# Stage 1 : Spectral peaks

In [28]:
import matplotlib.pyplot as plt
import librosa.display

import numpy as np
import pandas as pd
import librosa

In [10]:
def find_spectral_peaks(filename):
    y, sr = librosa.load(filename)
    D = np.abs(librosa.stft(y))
    tst_array = D
    max_filter_size = [91, 65]
    for x_init in range(0, tst_array.shape[0], max_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], max_filter_size[1]):
            subarr = tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])]
            mx = np.amax(subarr)
            tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])] = np.where(subarr != mx, 0, subarr)
    # print("MAX FILTER APPLIED:")
    min_filter_size = [3, 3]
    for x_init in range(0, tst_array.shape[0], min_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], min_filter_size[1]):
            subarr = tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])]
            mn = np.amin(subarr)
            tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])] = np.where(subarr != mn, subarr, 0)
    # print("MIN FILTER APPLIED:")
    return np.count_nonzero(tst_array), D

In [11]:
def get_peak_coordinates_from_matrix(matrix):
    coordinates = []
    for row in matrix:
        for col in row:
            if matrix[row][col] != 0:
                coordinates.append((row, col))
    return coordinates

# Stage 2 : Storing hashes

In [30]:
import sqlite3
from rtree import index

In [31]:
conn = sqlite3.connect('quad.db')
print("Opened database successfully")

Opened database successfully


In [32]:
conn.execute('''CREATE TABLE UNNORMALIZED_QUADS
         (ID INT PRIMARY KEY     NOT NULL AUTOINCREMENT,
         AX  INT                 NOT NULL,
         AY  INT                 NOT NULL,
         BX  INT                 NOT NULL,
         BY  INT                 NOT NULL,
         CX  INT                 NOT NULL,
         CY  INT                 NOT NULL,
         DX  INT                 NOT NULL,
         DY  INT                 NOT NULL);''')


In [33]:
conn.execute('''CREATE TABLE FIDINDEX
         (FID INT PRIMARY KEY    NOT NULL,
         FILENAME TEXT           NOT NULL,
         QUADIDX_START INT       NOT NULL,
         QUADIDX_END   INT       NOT NULL);''')

In [14]:
# TODO : Complete this function
def quads_sanity_check(quads):
    return 1

In [29]:
def db_insert_quads_record(file, quads):
    if quads_sanity_check(quads) == 0:
        return
    cur = conn.cursor()
    cur.execute("SELECT ID FROM UNNORMALIZED_QUADS ORDER BY ID DESC LIMIT 1")
    row = cur.fetchall()
    if not row:
        start_id = 0
    start_id = 200
    for a in quads:
        cur.execute("INSERT INTO UNNORMALIZED_QUADS VALUES ({}, {}, {}, {}, {}, {}, {}, {}, {})".format(
                    (start_id+1), a[0][0], a[0][1], a[1][0], a[1][1], a[2][0], a[2][1], a[3][0], a[3][1]))
        start_id += 1
    return row

# System simulations

In [37]:
import os
import time
path = '/home/supraraj/Projects/AudioFinger/audio_files/'

audio_input_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mp3' in file:
            audio_input_files.append(os.path.join(r, file))

In [30]:
for file in audio_input_files:
    print("")
    print("File: {}".format(file))
    start = time.time()
    #peak_count, peaks = find_spectral_peaks(file)
    #peaks_coordinates = get_peak_coordinates_from_matrix(peaks)
    
    # expected to get a list of tuples of the form [[(x,y), (bx_off,by_off), (cx_off,cy_off), (dx_off,dy_off)], ...] 
    quads = [[(3,4), (10, 40), (2, 5), (3, 15)],
                 [(6,7), (8, 23), (1, 1), (2, 2)]]
    
    row = db_insert_quads_record(file, quads)
    print(row)
    end = time.time()
    print("Peak count: {} in time {}".format(peak_count, (end-start)))
        


File: /home/supraraj/Projects/AudioFinger/audio_files/Nico & Vinz - Am I Wrong [Official Music Video].mp3
[(2,)]
Peak count: 2388 in time 0.0007584095001220703

File: /home/supraraj/Projects/AudioFinger/audio_files/P!nk - Just Give Me A Reason ft. Nate Ruess (1).mp3


IntegrityError: UNIQUE constraint failed: UNNORMALIZED_QUADS.ID